In [23]:
# ==============================
# Brain Tumor Detection using Hugging Face + Gradio
# ==============================
!pip install torch torchvision transformers gradio --quiet

import torch
from transformers import AutoModelForImageClassification, AutoImageProcessor
from PIL import Image
import gradio as gr

# Load pre-trained model and processor
model_name = "ShimaGh/Brain-Tumor-Detection"
model = AutoModelForImageClassification.from_pretrained(model_name)
processor = AutoImageProcessor.from_pretrained(model_name)

# Define prediction function
def predict(image):
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_idx = logits.argmax(-1).item()
    class_names = ["No Tumor", "Tumor"]  # adjust based on model's labels
    predicted_class = class_names[predicted_class_idx]
    return predicted_class

# Gradio interface
iface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="Upload MRI Image"),
    outputs=gr.Textbox(label="Prediction"),
    title="🧠 Brain Tumor Detection",
    description="Upload an MRI image to classify whether it contains a brain tumor.",
    examples=[]  # optional: add example images
)

iface.launch(share=True)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/347M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://299463e74930424fbd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [32]:
# ==============================
# Brain Tumor Detection with Login/Register (SQLite + Gradio)
# ==============================
!pip install torch torchvision transformers gradio --quiet

import sqlite3
import torch
from transformers import AutoModelForImageClassification, AutoImageProcessor
from PIL import Image
import gradio as gr

# -----------------------------
# SQLite setup
# -----------------------------
DB_PATH = "users.db"

def init_db():
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS users
                 (username TEXT PRIMARY KEY, password TEXT)''')
    conn.commit()
    conn.close()

def register_user(username, password):
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    try:
        c.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, password))
        conn.commit()
        return "✅ Registration successful! Please login."
    except sqlite3.IntegrityError:
        return "❌ Username already exists!"
    finally:
        conn.close()

def login_user(username, password):
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    c.execute("SELECT * FROM users WHERE username=? AND password=?", (username, password))
    user = c.fetchone()
    conn.close()
    if user:
        return True, "✅ Login successful!"
    else:
        return False, "❌ Invalid username or password!"

# Initialize database
init_db()

# -----------------------------
# Load Brain Tumor Model
# -----------------------------
model_name = "ShimaGh/Brain-Tumor-Detection"
model = AutoModelForImageClassification.from_pretrained(model_name)
processor = AutoImageProcessor.from_pretrained(model_name)

def predict(image):
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_idx = logits.argmax(-1).item()
    class_names = ["No Tumor", "Tumor"]
    predicted_class = class_names[predicted_class_idx]
    return predicted_class

# -----------------------------
# Gradio Blocks
# -----------------------------
with gr.Blocks() as app:

    # Login/Register Section
    with gr.Column(visible=True) as login_col:
        gr.Markdown("## 🧠 Brain Tumor Detection - Login/Register")
        username_input = gr.Textbox(label="Username")
        password_input = gr.Textbox(label="Password", type="password")
        login_btn = gr.Button("Login")
        register_btn = gr.Button("Register")
        login_msg = gr.Textbox(label="Message", interactive=False)

    # Prediction Section (Hidden by default)
    with gr.Column(visible=False) as pred_col:
        gr.Markdown("## Upload MRI Image for Brain Tumor Detection")
        image_input = gr.Image(type="pil")
        predict_btn = gr.Button("Predict")
        output_text = gr.Textbox(label="Prediction", interactive=False)
        logout_btn = gr.Button("Logout")

    # -----------------------------
    # Button actions
    # -----------------------------
    def login_action(username, password):
        success, msg = login_user(username, password)
        if success:
            return gr.update(visible=False), gr.update(visible=True), msg
        else:
            return gr.update(visible=True), gr.update(visible=False), msg

    login_btn.click(login_action, inputs=[username_input, password_input],
                    outputs=[login_col, pred_col, login_msg])

    register_btn.click(register_user, inputs=[username_input, password_input], outputs=[login_msg])

    predict_btn.click(predict, inputs=[image_input], outputs=[output_text])

    logout_btn.click(lambda: (gr.update(visible=True), gr.update(visible=False), "🔒 Logged out successfully!"),
                     outputs=[login_col, pred_col, login_msg])

# Launch app
app.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://53c2b82b219aa9bb0f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
